<a href="https://colab.research.google.com/github/Julita257/UMwF/blob/Case-2/Case_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Przegląd danych:

*   Podaj liczbę filmów Sci-Fi
*   Pokaż rozkład ocen komedii
*   Podaj średnią ocen wszystkich filmów akcji oraz 3 filmy najwyżej ocenianych????? ***(Dopytać się)***

2. System rekomendacyjny

*  Zbuduj system na podstawie algorytmu SVD oraz kNNwithMeans
*  Czym różni się algorytm kNN with means od standardowego kNN?
*  Wykorzystaj metodę hiperparametryzacji GridSearch do wyboru liczby sąsiadów od 2-6
*  W ocenie metod wykorzystaj walidację krzyżową
3. Podaj rekomendacje po obejrzeniu filmu: Jumanji  oraz Flint

1. Dane

In [3]:
import pandas as pd


In [7]:
ratings = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv')
movies = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv')

In [8]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Sprawdź:
- Liczbę wszystkich ocen
- Liczbę filmów w bazie
- Liczbę użytkowników
- Średnią liczbę ocen na użytkownika
- Średnią liczbę ocen na film
- Rozkład ocen

In [19]:
all_scores = ratings['rating'].count()
all_scores

100836

In [22]:
movie_count = movies['movieId'].count()
movie_count

9742

In [24]:
user_count = len(ratings['userId'].unique())
user_count

610

In [25]:
AVG_score_user = all_scores / user_count
AVG_score_user

165.30491803278687

In [27]:
AVG_score_movie = all_scores / movie_count
AVG_score_movie

10.350646684459043

In [29]:
combined_data = pd.merge(ratings, movies, on='movieId')
combined_data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [34]:
sci_fi_movies = combined_data[combined_data['genres'].str.contains('Sci-Fi')]
sci_fi_count = sci_fi_movies['movieId'].nunique()

In [36]:
print(f'Liczba filmów Sci-Fi: {sci_fi_count}')

Liczba filmów Sci-Fi: 980


In [41]:
comedy_movies = combined_data[combined_data['genres'].str.contains('Comedy')]
comedy_distribution = comedy_movies['rating'].value_counts(normalize=True).sort_index()

In [42]:
comedy_distribution

rating
0.5    0.016183
1.0    0.033723
1.5    0.022918
2.0    0.087189
2.5    0.064784
3.0    0.212685
3.5    0.130233
4.0    0.247331
4.5    0.071544
5.0    0.113410
Name: proportion, dtype: float64

In [44]:
action_movies = combined_data[combined_data['genres'].str.contains('Action')]
AVG_score_action = action_movies['rating'].mean()
AVG_score_action

3.447984331646809

2. System rekomendacyjny

In [46]:
! pip install scikit-surprise
from surprise import Dataset, Reader
from surprise import SVD, KNNWithMeans
from surprise.model_selection import cross_validate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162996 sha256=0e3a77a4d51acc8b5653f86254eeaeba690dc47c7a49124e3b0e678c53a1593a
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise
